In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer


paths = [str(x) for x in Path(".").glob("../data/whole-data/*")]

# ტოკენაიზერის შექმნა
tokenizer = ByteLevelBPETokenizer()
# უნიკალური ტოკენების რაოდენობა - 15 000
tokenizer.train(files=paths, vocab_size=15_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


creating tokenizer


In [3]:
import os
if not os.path.exists('../resources/roberta'):
    os.makedirs('../resources/roberta')

tokenizer.save_model("../resources/roberta")

['resources/vocab.json', 'resources/merges.txt']

In [4]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=15_000,
    max_position_embeddings=514,
    num_attention_heads=2,
    num_hidden_layers=2,
    type_vocab_size=1,
)

In [5]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("../resources/roberta", max_len=512)

In [6]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [7]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="../data/roberta-corpus/corpus.txt",
    block_size=128,
)

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:114: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 6min 20s, sys: 6.44 s, total: 6min 26s
Wall time: 3min 53s


In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="../resources/roberta",
    overwrite_output_dir=True,
    num_train_epochs=6,
    per_gpu_train_batch_size=64,
    save_steps=10_000, # ყოველი 10 000 სთეფის მერე ინახავს მოდელს
    save_total_limit=2, # ჯამში 2 საუკეთესო მოდელს ინახავს
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/opt/conda/lib/python3.7/site-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [10]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,8.114117
1000,7.705393
1500,7.479248
2000,7.335984
2500,7.219426
3000,7.106988
3500,7.007855
4000,6.877320
4500,6.755910
5000,6.616125


CPU times: user 4h 58min 12s, sys: 1min 9s, total: 4h 59min 22s
Wall time: 5h 3min 22s


TrainOutput(global_step=79740, training_loss=4.193962800978179)

In [11]:
trainer.save_model("../resources/roberta")